In [3]:
# 异步线程＋回调函数

from threading import Thread
import time
import random



def download(url, method='GET', data=None, callback=None):
    print('正在打开', url,method, sep='->')
    time.sleep(random.uniform(5, 10))
    print('下载完成', url)
    callback(('我是下载的内容...', 200))
    

def get(url,callback):
    # 启动线程，执行下载
    t = Thread(target=download,
               kwargs={'url':url,'callback': callback})
    t.start()
    print('---正在下载,请等待---')
    
def parse(response):
    print('---下载之后的解析处理---')
    print(response)
    
get('http://www.jd.com', callback=parse)

正在打开---正在下载,请等待---
->http://www.jd.com->GET
下载完成 http://www.jd.com
---下载之后的解析处理---
('我是下载的内容...', 200)


In [8]:
import time

# 装饰函数： 用于性能监测， 日志， 权限验证
def check_runtime(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # 获取执行目标函数之前的时间戳
        # 执行目标函数
        result = func(*args, **kwargs)
        
        # 计算函数执行的时间（秒）
        delta_time = time.time() - start_time
        print('执行', func.__name__, '所需要的时间(秒):', delta_time)
        return result
    return wrapper
    
@check_runtime
def downloader(url):
    print('开始下载', url)
    time.sleep(5)
    print('下载完成')
    
downloader('www.jd.com')

开始下载 www.jd.com
下载完成
执行 downloader 所需要的时间(秒): 5.004284143447876


In [34]:
import random

# 模拟验证 当前用户是否拥有 申请(上报- SQ)的权限
# 声明有参数的装饰函数(三层， 最外层接收参数， 后二层按普通装修函数设计)
def check_qx(qxName):
    print('---a--', qxName)
    def wrapper1(func):
        print('--b---', qxName)
        def wrapper2(*args, **kwargs):
            # 执行目标函数之前，需要验证当前用户是否有qxName权限
            # 用户表中： role_id 
            # 角色表中： qx_rights(权限值)
            # 权限表：  qx_id, qx_name, qx_right(权限值)
            rights = random.randint(1,10)
            print('---c---',qxName,rights)
            if (rights >= 6 and qxName== 'SQ') or\
               (rights >= 5 and qxName=='DEL_USER'):
                result = func(*args, **kwargs)
                return result
            else:
                print('对不起，您暂无权限', qxName)
        return wrapper2
    return wrapper1
                
@check_qx('SQ')
@check_qx('DEL_USER')
def post_sq(url, data):
    print('正在上报', url, data)
    

def del_user(user_id):
    print('正在删除用户', user_id)
    
post_sq('www.baidu.com', 'disen')
# del_user(1)

---a-- SQ
---a-- DEL_USER
--b--- DEL_USER
--b--- SQ
---c--- SQ 6
---c--- DEL_USER 7
正在上报 www.baidu.com disen
